In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%load_ext autotime

import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

df_dis = pd.read_csv('riyadh_resturants_clean.csv')

time: 1.07 s (started: 2021-11-14 00:23:52 +03:00)


# Drop all Un-necessary data:

In [2]:
# Drop all null values in price column
df_dis.dropna(subset=['price'],inplace=True)
df_dis.price.isna().sum()

0

time: 75.6 ms (started: 2021-11-14 00:23:53 +03:00)


In [3]:
# Drop all Un-necessary categories
df_dis = df_dis[~ df_dis.categories.str.contains('Molecular')]
df_dis = df_dis[~ df_dis.categories.str.contains('Deli')]
df_dis = df_dis[~ df_dis.categories.str.contains('Diner')]
df_dis = df_dis[~ df_dis.categories.str.contains('Dim')]
df_dis = df_dis[~ df_dis.categories.str.contains('Australian')]
df_dis = df_dis[~ df_dis.categories.str.contains('Soup')]
df_dis = df_dis[~ df_dis.categories.str.contains('Gastropub')]
df_dis = df_dis[~ df_dis.categories.str.contains('Persian')]
df_dis = df_dis[~ df_dis.categories.str.contains('Scandinavian')]
df_dis = df_dis[~ df_dis.categories.str.startswith('Restaurant')]

time: 186 ms (started: 2021-11-14 00:23:53 +03:00)


# Handle the Inconsistent categories:

__Here I recategoraized the data based on category type and their names.__

In [4]:
df_dis.loc[df_dis.name.str.contains('saadeddin')|df_dis.name.str.contains('سعد الدين')|df_dis.name.str.contains('sweet')|
df_dis.categories.str.contains('essert')|df_dis.name.str.contains('حلوي')|df_dis.name.str.contains('حلى')|df_dis.name.str.contains('شوكولات')|
df_dis.categories.str.contains('Donut') | df_dis.categories.str.contains('Cupcake'), 'categories'] = "Dessert Shop" 

time: 121 ms (started: 2021-11-14 00:23:54 +03:00)


In [5]:
df_dis.loc[df_dis.name.str.contains('hawarma')|df_dis.categories.str.contains('hawarma')|df_dis.name.str.contains('شاورما'),'categories'] = 'Shawarma Restaurant'

time: 44.7 ms (started: 2021-11-14 00:23:54 +03:00)


In [6]:
df_dis.loc[df_dis.categories.str.contains('offee')|df_dis.categories.str.contains('Caf')|df_dis.name.str.contains('كافي') |
df_dis.categories.str.contains('Tea'), 'categories'] = "Cafe"

time: 56.1 ms (started: 2021-11-14 00:23:54 +03:00)


In [7]:
df_dis.loc[df_dis.categories.str.contains('uice')|df_dis.categories.str.contains('rink')|df_dis.name.str.contains('عص'), 'categories'] = "Drink Shop"

time: 39.6 ms (started: 2021-11-14 00:23:54 +03:00)


In [8]:
df_dis.loc[ df_dis.categories.str.contains('Sandwich') | df_dis.name.str.contains('andwich') | 
df_dis.categories.str.contains('Breakfast') | df_dis.categories.str.contains('Pie'), 'categories'] = "Sandwich Restaurant"

time: 42.8 ms (started: 2021-11-14 00:23:54 +03:00)


In [9]:
df_dis.loc[df_dis.categories.str.contains('merican')| df_dis.categories.str.contains('urger')| df_dis.name.str.contains('urger')| 
df_dis.name.str.contains('برجر')| df_dis.categories.str.contains('Fast')| df_dis.categories.str.contains('Wings')| 
df_dis.categories.str.contains('Fried Chicken')| df_dis.categories.str.contains('Mac & Cheese Joint')|
df_dis.categories.str.contains('Hot Dog Joint') , 'categories'] = 'Fast Food Restaurant'

time: 109 ms (started: 2021-11-14 00:23:54 +03:00)


In [10]:
df_dis.loc[df_dis.categories.str.contains('Ice')|df_dis.categories.str.contains('ream') |df_dis.categories.str.contains('Frozen Yogurt') |
df_dis.name.str.contains('كريم'), 'categories'] = "Ice Cream Shop"

time: 50.2 ms (started: 2021-11-14 00:23:54 +03:00)


In [11]:
df_dis.loc[ df_dis.categories.str.contains('talian')| df_dis.categories.str.contains('izza')| df_dis.name.str.contains('بيتزا'), 'categories'] = 'Italian Restaurant'

time: 37.2 ms (started: 2021-11-14 00:23:54 +03:00)


In [12]:
df_dis.loc[df_dis.categories.str.contains('eafood')| df_dis.categories.str.contains('Fish')|df_dis.categories.str.contains('fish')| 
df_dis.categories.str.contains('Sushi') | df_dis.name.str.contains('سمك'), 'categories'] = 'Seafood Restaurant'

time: 52.7 ms (started: 2021-11-14 00:23:55 +03:00)


In [13]:
df_dis.loc[ df_dis.categories.str.contains('Asian')|df_dis.categories.str.contains('Middle Eastern Restaurant')| 
df_dis.categories.str.contains('Falafel') | df_dis.categories.str.contains('Southern') ,'categories'] = 'Arabic Restaurant'

time: 49.4 ms (started: 2021-11-14 00:23:55 +03:00)


In [14]:
df_dis.loc[ df_dis.name.str.contains('hinese')| df_dis.categories.str.contains('Noodle')| df_dis.categories.str.contains('hinese')| 
df_dis.categories.str.contains('Jap')|df_dis.categories.str.contains('Filipino') |df_dis.categories.str.contains('Vietnamese')| df_dis.categories.str.contains('Malay')|
df_dis.categories.str.contains('Indonesian') |df_dis.categories.str.contains('Korean')|df_dis.categories.str.contains('Mongolian') | df_dis.categories.str.contains('Thai') | 
df_dis.name.str.contains('اسيوي'), 'categories'] = 'Asian Restaurant'

time: 160 ms (started: 2021-11-14 00:23:55 +03:00)


In [15]:
df_dis.loc[df_dis.categories.str.contains('BBQ')|df_dis.categories.str.contains('Lebanese')| df_dis.categories.str.contains('Steakhouse')|
df_dis.categories.str.contains('Kebab')|df_dis.name.str.contains('لبنان'), 'categories'] = 'BBQ Restaurant'

time: 58.7 ms (started: 2021-11-14 00:23:55 +03:00)


In [16]:
df_dis.loc[ df_dis.categories.str.contains('Bakery')| df_dis.categories.str.contains('Bagel')|
df_dis.categories.str.contains('Pastry')|df_dis.name.str.contains('خابز'), 'categories'] = 'Bakery Shop'

time: 40.4 ms (started: 2021-11-14 00:23:55 +03:00)


In [17]:
df_dis.loc[df_dis.name.str.contains('خضار')|df_dis.name.str.contains('زراعي') |
df_dis.categories.str.contains('Vegetarian / Vegan Restaurant') | df_dis.name.str.contains('بنده'), 'categories'] = 'Grocery Shop'

time: 42.5 ms (started: 2021-11-14 00:23:55 +03:00)


In [18]:
df_dis.loc[ (df_dis.categories.str.contains('Vegetarian')|df_dis.categories.str.contains('Vegan')|df_dis.categories.str.contains('Vegetarian / Vegan') |df_dis.categories.str.contains('Salad')|
df_dis.categories.str.contains('Health')| df_dis.categories.str.contains('Snack')|df_dis.categories.str.contains('Gluten') ) 
& ~ df_dis.name.str.contains('هايبر'), 'categories'] = 'Healthy Food Restaurant'

time: 92.7 ms (started: 2021-11-14 00:23:56 +03:00)


In [19]:
df_dis.loc[ df_dis.categories.str.contains('Food Truck')| df_dis.categories.str.contains('Truck')| 
df_dis.name.str.contains('فود ترك')| df_dis.name.str.contains('Truck'), "categories"] = 'Food Truck'

time: 41.5 ms (started: 2021-11-14 00:23:56 +03:00)


In [20]:
df_dis.loc[ df_dis.categories.str.contains('Lounge') | df_dis.name.str.contains('Lounge'), "categories"] = 'Lounge Restaurant'

time: 28.8 ms (started: 2021-11-14 00:23:56 +03:00)


In [21]:
df_dis.loc[ df_dis.categories.str.contains('Mexican')| df_dis.categories.str.contains('Burrito') |df_dis.name.str.contains('Taco') | 
df_dis.categories.str.contains('Taco'), "categories"] = 'Mexican Restaurant'

time: 45 ms (started: 2021-11-14 00:23:56 +03:00)


In [22]:
df_dis.loc[df_dis.categories.str.contains('Afghan') | df_dis.name.str.contains('فلافل')| df_dis.categories.str.contains('Dumpling') | 
df_dis.categories.str.contains('Arepa') , "categories"] = 'Arabic Restaurant'

time: 44.1 ms (started: 2021-11-14 00:23:56 +03:00)


In [23]:
df_dis.loc[ df_dis.categories.str.contains('Indian') | df_dis.name.str.contains('هندي'), 'categories'] = 'Indian Restaurant'

time: 27.4 ms (started: 2021-11-14 00:23:57 +03:00)


In [24]:
df_dis.loc[df_dis.categories.str.contains('African')| df_dis.categories.str.contains('Egypt')| df_dis.categories.str.contains('Ethiopian')|df_dis.categories.str.contains('Moroccan')| 
df_dis.name.str.contains('افريقي')| df_dis.name.str.contains('مصري')| df_dis.name.str.contains('سوداني'), 'categories'] = 'African Restaurant'

time: 111 ms (started: 2021-11-14 00:23:57 +03:00)


In [25]:
df_dis.loc[ df_dis.categories.str.contains('Turkish') | 
df_dis.categories.str.contains('Swiss')  |
df_dis.categories.str.contains('Portuguese') |
df_dis.categories.str.contains('Spanish') | 
df_dis.categories.str.contains('Tapas') |
df_dis.categories.str.contains('German') |
df_dis.categories.str.contains('European') |
df_dis.categories.str.contains('Mediterranean') |
df_dis.categories.str.contains('Armenian') |
df_dis.categories.str.contains('Greek'), 'categories'] = 'European Restaurant'

time: 140 ms (started: 2021-11-14 00:23:57 +03:00)


In [26]:
df_dis.loc[ df_dis.categories.str.contains('Peruvian')  |
df_dis.categories.str.contains('Portuguese') |
df_dis.categories.str.contains('Argentinian') |
df_dis.categories.str.contains('Brazilian') |
df_dis.categories.str.contains('Caribbean') |
df_dis.categories.str.contains('Mediterranean') |
df_dis.categories.str.contains('Cuban') |
df_dis.categories.str.contains('Caucasian') , 'categories'] = 'American Restaurant'

time: 215 ms (started: 2021-11-14 00:23:57 +03:00)


# How many category/rows we have now?

In [27]:
df_dis.categories.nunique()

23

time: 7.87 ms (started: 2021-11-14 00:23:58 +03:00)


In [28]:
df_dis.categories.unique()

array(['Arabic Restaurant', 'Cafe', 'Dessert Shop', 'Ice Cream Shop',
       'Fast Food Restaurant', 'Sandwich Restaurant',
       'Italian Restaurant', 'Bakery Shop', 'Drink Shop',
       'Seafood Restaurant', 'BBQ Restaurant', 'European Restaurant',
       'Indian Restaurant', 'Food Truck', 'Asian Restaurant',
       'Healthy Food Restaurant', 'Shawarma Restaurant', 'Grocery Shop',
       'African Restaurant', 'American Restaurant', 'Mexican Restaurant',
       'French Restaurant', 'Lounge Restaurant'], dtype=object)

time: 5.85 ms (started: 2021-11-14 00:23:58 +03:00)


In [29]:
df_dis.shape

(16879, 11)

time: 4.7 ms (started: 2021-11-14 00:23:58 +03:00)


# Export the handled dataset for continue project:

In [30]:
df_dis.to_excel("handled_dataset.xlsx")

time: 5.5 s (started: 2021-11-14 00:29:58 +03:00)
